In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from huggingface_hub import login

login("hf_token")  # Replace with your actual key


In [ ]:
!pip install -q transformers accelerate einops sentencepiece bitsandbytes

In [ ]:
!pip install -q transformers accelerate bitsandbytes peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 27.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers accelerate --quiet

In [ ]:
!pip install transformers peft accelerate bitsandbytes datasets --quiet

In [ ]:
!pip install trl

In [ ]:
!pip install bitsandbytes
!pip install accelerate


In [ ]:
pip list


In [ ]:
!nvidia-smi


Wed Jun 11 08:07:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
pip install --upgrade bitsandbytes --extra-index-url https://pypi.org/simple


In [ ]:
import torch
import bitsandbytes as bnb
print("Torch CUDA version:", torch.version.cuda)
print("BitsAndBytes version:", bnb.__version__)


Torch CUDA version: 12.4
BitsAndBytes version: 0.46.0


In [ ]:
!pip uninstall -y bitsandbytes
!pip install --no-cache-dir bitsandbytes


In [ ]:
pip install --upgrade torch transformers bitsandbytes accelerate


In [ ]:
# ⬆️ Upgrade bitsandbytes and install dependencies
!pip install -U bitsandbytes
!pip install -U transformers accelerate datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set save path in your Google Drive
save_path = "/content/drive/MyDrive/distilgpt2"

# Load tokenizer and model
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Save tokenizer and model to Google Drive
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

print(f"Model and tokenizer saved to {save_path}")



In [ ]:
#main
import os
import time
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from datasets import Dataset

# ✅ Paths
model_name = "distilgpt2"
train_csv = "/content/drive/MyDrive/llm-classification-finetuning/train.csv"
output_dir = "/content/drive/MyDrive/llm-classification-finetuning/distilgpt2-finetuned"

# ✅ Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# ✅ Load model
model = AutoModelForCausalLM.from_pretrained(model_name)

# ✅ Load and sample dataset
df = pd.read_csv(train_csv)
df = df.sample(n=500, random_state=42)  # ← Adjust as needed for faster runs

# ✅ Prompt formatting
def format_prompt(batch):
    texts = []
    for prompt, resp_a, resp_b, winner_a, winner_b in zip(
        batch["prompt"], batch["response_a"], batch["response_b"],
        batch["winner_model_a"], batch["winner_model_b"]
    ):
        prompt_text = f"""A user asked the following question:
"{prompt}"

Two different AI assistants replied:

Response A:
"{resp_a}"

Response B:
"{resp_b}"

Which response is more helpful, human-like, and aligned with user expectations? Reply with only "A" or "B".
"""
        label = "A" if winner_a == 1 else ("B" if winner_b == 1 else "C")
        texts.append(prompt_text + label)
    return {"text": texts}

# ✅ Hugging Face dataset + formatting
dataset = Dataset.from_pandas(df)
dataset = dataset.map(format_prompt, batched=True)

# ✅ Tokenization
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256,
    )

tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names, batched=True)

# ✅ Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# ✅ Training arguments with checkpoint saving
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=46,
    logging_steps=50,
    save_strategy="steps",       # Save regularly for resuming
    save_steps=500,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    report_to="none",
    push_to_hub=False,
)

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ✅ Resume from checkpoint if available
last_checkpoint = None
if os.path.isdir(output_dir):
    checkpoints = [os.path.join(output_dir, d) for d in os.listdir(output_dir) if d.startswith("checkpoint")]
    if checkpoints:
        last_checkpoint = max(checkpoints, key=os.path.getmtime)
        print(f"🔁 Resuming from checkpoint: {last_checkpoint}")

# ✅ Train
start = time.time()
trainer.train(resume_from_checkpoint=last_checkpoint)
print(f"⏱️ Training finished in {(time.time() - start)/60:.2f} minutes")

# ✅ Save final model
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Finetuning complete — model saved to Google Drive.")


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

<ipython-input-7-748452056>:90: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🔁 Resuming from checkpoint: /content/drive/MyDrive/llm-classification-finetuning/distilgpt2-finetuned/checkpoint-368


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Step,Training Loss
400,0.816200
450,0.711700
500,0.607100
550,0.523800
600,0.465700
650,0.423200
700,0.395600


⏱️ Training finished in 3.45 minutes
✅ Finetuning complete — model saved to Google Drive.
